pro eclipse,P,ap,Rplan,inc,obli=obli,rot=rot,wl,dt,ts,lc,fa,ffi,longs,slat=slat,star=star,notransit=notransit,plot=plot,silent=silent

; PURPOSE:
;	Calculate the light curve of a planetary eclipse of its parent
; star. With spots added.
;
; INPUT:
;	ap = semi-major axis of planetary orbit (in Rstar)
;	P  = Period of planetary orbit (in days), assumed circular orbit
; 	Rplan = planetary radius in Rstar (1 Rjup=6.9911e4 km=0.100447 Rsun)
;	inc  = inclination angle of orbital plane
;	wl = white light image of star (or Sun)
;	dt = time interval between light curve points (in minutes)
;       obli = obliquiti angle (in degrees) (default = 0)
; SPOTS: if more than one the variables below are vectors
;	fa = spot size in units of planet radius, Rp
;	fi = spot intensity (fraction of central brightness, fi=1 means no spot)
;	longs = spot longitude position (in degrees)
;	slat  = latitude of spot with respect to stellar equator (in degrees), 
;               default is spots on latitude of transit crossing
; OUTPUT:
;	ts  = time array (in hours w.r.t. transit central time)
;	lc = light curve (dL/L)
; KEYWORD:
;	star = image of the star with spots
; 	notransit = only returns star, not the light-curve


	n=n_elements(wl[0,*])
	mx=mean(wl[n/2-20:n/2+20,n/2-20:n/2+20])
	half,findgen(n),wl[*,n/2],x1,x2,yval=mx/2.
	r=(x2-x1)/2.		; raio da estrela em pixels
	
Rs=Rplan*r	; raio do planeta
a=ap*r		; raio orbital (Rstar)
alfa=inc*!dtor

if(ap eq 0. or Rplan eq 0.) then return

teta=findgen(P*24.*dt+0.5)*360./(P*24*dt)

ii=where(teta ge 180. and teta le 360.)

teta=teta[ii]*!dtor

xx=a*cos(teta)
yy=a*sin(teta)*cos(alfa)

if(keyword_set(obli) eq 0) then obli=0. else obli=obli
if(keyword_set(rot) eq 0) then rot=0. else rot=rot

;xxp=xx*cos(obli*!dtor)-yy*sin(obli*!dtor)
;yyp=xx*sin(obli*!dtor)+yy*cos(obli*!dtor)

xxp=xx
yyp=yy

pp=where(abs(xxp) lt n/2*1.2 and abs(yyp) lt n/2,np)

xp=xxp[pp]
yp=yyp[pp]

jj=sort(xp)
xp=xp[jj]
yp=yp[jj]

if(inc gt 90.) then yp=-yp

wp=fltarr(np)+total(wl)

if(mean(abs(yp[pp]/r)) gt 1) then begin
   if(keyword_set(silent) eq 0) then $
	print,'Planet does not eclipse star (increase inc)'
   return
endif

; latitude of projected transit, for zero obliquity
lat=-asin(ap*cos(inc*!dtor))/!dtor ; latitude Sul (arbitraria)

; Starspots

jj=where((abs(longs) lt 90) or (longs gt 270 and longs lt 360.),nspot)
if(keyword_set(slat) eq 0) then slat=replicate(-asin(ap*cos(alfa))/!dtor,nspot) else slat=slat[jj]

lgs=longs[jj]
fi=ffi[jj]

Ra=Rs*fa[jj]
ys=r*sin(slat*!dtor)

; longitude of spots:
xs=r*cos(slat*!dtor)*sin(lgs*!dtor)
slg=asin(xs/r)	; spot longitude
helio=acos(cos(slat*!dtor)*cos(slg))/!dtor

spot=fltarr(n,n)+1.
; efeito de foreshortenning (4/7/2008)

kk=lindgen(n*n)

for i=0,nspot-1 do $
if(xs[i] lt r and ys[i] lt r and Ra[i]*cos(helio[i]*!dtor) ge 1) then begin	; para cada mancha
tmp=fltarr(n,n)+1.

yy=ys[i]+n/2
xx=xs[i]+n/2
ii=where((kk/n-yy)^2+((kk-n*fix(kk/n)-xx)/cos(helio[i]*!dtor))^2 le Ra[i]^2)
tmp[ii]=0.

tmp2=rot(tmp,sign(slat[i],-slat[i]*slg[i]/!dtor),1,xs[i]+n/2.,ys[i]+n/2.,/pivot)
ii=where(tmp2 lt 1.)

spot[ii]=fi[i]
;spot=spot-1.+tmp2

endif

star=wl*spot

jj=where(xp+n/2 gt 0 and xp+n/2 lt n and yp+n/2 gt 0 and yp+n/2 lt n,nj) 
i0=jj[0]
i1=jj[nj-1]

if(keyword_set(notransit) eq 0) then $
   for i=i0,i1 do begin
	plan=fltarr(n,n)+1
        x0=xp[i]+n/2
        y0=yp[i]+n/2
        ii=where((kk/n-y0)^2+(kk-n*fix(kk/n)-x0)^2 le rs^2)
        plan[ii]=0.

	plan=smooth(plan,3)
  	wp[i]=total(star*plan)


if(keyword_set(plot) and i eq np/2) then begin
grayplot,star*plan,xar=findgen(n)-n/2,yar=findgen(n)-n/2
oplot,xp,yp
ang=findgen(181)*!dtor*2

coord,r,obli,rot

endif
   endfor

;lc=wp
lc=wp/total(star)<1

d=sqrt(xp^2+yp^2)
ii=where(d eq min(d))
ts=(findgen(np)-ii[0])/dt

end
